# Imports


In [1]:
#Data loading 
import torch
from torch.utils import data
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import random
import numpy

#model 
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

# 

# Creating dataloaders
Source : https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

## Definie the data encoder the data

In [2]:
#Data encoder 
# def sparse_encode(seq, rna_dict):
#     """For a given nucleotide sequence, return a sequence with each nucleotide encoded as a one hot vector"""
#     enc_seq = np.zeros([len(seq),len(RNA_DICT)])
#     for i, nuc in enumerate(seq):
#         enc_seq[i, rna_dict[nuc]] = 1
#     return enc_seq

# def encode_with_padding(seq, max_len, rna_dict):
#     seq_onehot = np.zeros([max_len, len(RNA_DICT)], dtype=np.uint8)
#     length = len(seq)
#     encoded_seq = sparse_encode(seq, rna_dict)
#     seq_onehot[:length,:] = encoded_seq
#     seq_onehot[length:,5:6] = 1
#     return(seq_onehot)



## Load and encode the data

In [3]:
VAL = "data/val_filtered.txt"
TRAIN = "data/train_filtered.txt"
TEST = "data/test_filtered.txt"

RNA_DICT = {'A': 0, 'T': 1, 'C': 2, 'G': 3, 'N': 4, '-': 5}
partition = {'train':[], 'validation':[], 'test':[]} #lists containing ids of the sequences 
labels = {} # {seq_id : class}
sequences = {} # {seq_id : encoded_seq}
index = 0
# max_len = 0



with open(VAL, 'r') as fIn : 
    for line in fIn : 
        #manage seq id 
        seq_id = "id_"+str(index)
        index +=1
        #read line in file 
        seq = line.replace(" ", "").strip().split(",")
        
        #search for max seq len
#         max_len = len(seq[0]) if len(seq[0])>max_len else max_len
        #encode sequence
        #encod_seq = sparse_encode(seq[0], RNA_DICT)
        #store data
        partition['validation'].append(seq_id)
        labels[seq_id]=int(seq[1])
        sequences[seq_id] = seq[0]


with open(TRAIN, 'r') as fIn : 
    for line in fIn : 
        #manage seq id 
        seq_id = "id_"+str(index)
        index +=1
        #read line in file 
        seq = line.replace(" ", "").strip().split(",")
#         max_len = len(seq[0]) if len(seq[0])>max_len else max_len
        #encode sequence
        #encod_seq = sparse_encode(seq[0], RNA_DICT)
        #store data
        partition['train'].append(seq_id)
        labels[seq_id]=int(seq[1])
        sequences[seq_id] = seq[0]
        
        
with open(TEST, 'r') as fIn : 
    for line in fIn : 
        #manage seq id 
        seq_id = "id_"+str(index)
        index +=1
        #read line in file 
        seq = line.replace(" ", "").strip().split(",")
#         max_len = len(seq[0]) if len(seq[0])>max_len else max_len
        #encode sequence
        #encod_seq = sparse_encode(seq[0], RNA_DICT)
        #store data
        partition['test'].append(seq_id)
        labels[seq_id]=int(seq[1])
        sequences[seq_id] = seq[0]

# for seq_id, seq in sequences.items():
#     sequences[seq_id] = encode_with_padding(seq, max_len, RNA_DICT)


list_IDs = partition['train'] + partition['validation'] + partition['test']


In [4]:
def filter_90_percents_size(sequences, quantile_to_keep = 0.9):
    seq_length = [len(i) for _,i in sequences.items()]
    max_length_to_keep = int(numpy.quantile(seq_length, quantile_to_keep))
    new_sequences = {}
    for key,value in sequences.items() : 
        if len(value)<=max_length_to_keep : 
            new_sequences[key] = value 
        else : 
            if key in partition['train'] : partition['train'].remove(key)
            if key in partition['validation'] : partition['validation'].remove(key)
            if key in partition['test'] : partition['test'].remove(key)
    return(new_sequences, max_length_to_keep)
    
def encode_with_padding(sequences, max_lenght): 
    sequences = {key : np.array([RNA_DICT[i] for i in value]+[5]*(max_lenght-len(value))) for key,value in sequences.items()}
    return(sequences)

sequences, max_lenght = filter_90_percents_size(sequences)
sequences = encode_with_padding(sequences, max_lenght)

print(sequences['id_0'], sequences['id_1'])

[2 2 2 ... 5 5 5] [0 1 3 ... 5 5 5]


In [5]:
print([len(v) for k,v in sequences.items()])

[6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 6246, 624

## Define the dataset class

In [6]:
class Dataset(data.Dataset):
  """Characterizes a dataset for PyTorch"""
  def __init__(self, list_IDs, labels, sequences):
        """Initialization"""
        self.labels = labels
        self.list_IDs = list_IDs
        self.sequences = sequences
        
  def __len__(self):
        """Denotes the total number of samples"""
        return len(self.list_IDs)

  def __getitem__(self, index):
        """Generates one sample of data"""
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = self.sequences[ID]
        Y = self.labels[ID]

        return X, Y

## Create training and validation Dataloader

In [7]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters
params = {'batch_size': BATCH_SIZE,
          'shuffle': True,
          'num_workers': 0}

params_valid = {'batch_size': BATCH_SIZE,
          'shuffle': True,
          'num_workers': 0}


training_set = Dataset(partition['train'], labels, sequences)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['validation'], labels, sequences)
validation_generator = data.DataLoader(validation_set, **params_valid)

test_set = Dataset(partition['test'], labels, sequences)
test_generator = data.DataLoader(validation_set, **params)



def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()


# Build the model

In [49]:
# define network

num_embeddings = 6
embedding_dim = 4
num_l1 = 20

num_out = 6

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(num_embeddings,embedding_dim)
        self.embedding_dim = embedding_dim
        self.l1 = Linear(in_features=embedding_dim*max_lenght, 
                            out_features=num_l1,
                            bias=False)
        self.l2 = Linear(in_features=num_l1, 
                            out_features=num_out,
                            bias=False)
    
    def forward(self, x): # x.size() = [batch_size,max_length]
        x = x.unsqueeze_(0) # [1, batch_size,max_length]
        x = self.embedding(x) # [1, batch_size,max_length, embedding_dim]
        x = torch.squeeze(x) # [batch_size,max_length, embedding_dim]
        x = x.view(x.shape[0],-1)# [batch_size,seq_len = embedding_dim*max_lenght]
        x = relu(self.l1(x))
        x = self.l2(x)
        return x


net = Net()
if use_cuda:
    net.cuda()
print(net)

Net(
  (embedding): Embedding(6, 4)
  (l1): Linear(in_features=24984, out_features=20, bias=False)
  (l2): Linear(in_features=20, out_features=6, bias=False)
)


# Build the cost function

In [50]:
LEARNING_RATE = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

def accuracy(ys, ts):
    predictions = torch.max(ys, 1)[1]
    correct_prediction = torch.eq(predictions, ts)
    return torch.mean(correct_prediction.float())

# Test network

In [51]:
from random import randint 
#Test the forward pass with dummy data

def randnorm(size):
    return np.random.randint(0, 5, size).astype('long')

x = Variable(torch.from_numpy(randnorm((params["batch_size"], max_lenght))))
net(x)

torch.LongTensor
tensor([[3, 1, 2,  ..., 0, 1, 3],
        [3, 2, 3,  ..., 2, 2, 2],
        [1, 0, 1,  ..., 2, 0, 1],
        ...,
        [3, 1, 0,  ..., 4, 4, 2],
        [0, 2, 0,  ..., 3, 2, 2],
        [1, 4, 0,  ..., 2, 2, 2]])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])


tensor([[-0.2156, -0.3266,  0.1554, -0.1164, -0.3190, -0.0691],
        [-0.1681,  0.2186, -0.0374, -0.2055, -0.3995,  0.1048],
        [-0.4815, -0.1775,  0.0862,  0.2115, -0.0196,  0.2472],
        [-0.1669,  0.1545,  0.2346, -0.0432, -0.2386,  0.1332],
        [-0.1525, -0.0121,  0.0162, -0.0453, -0.2747,  0.2787],
        [-0.0635, -0.1472, -0.1171,  0.1154, -0.1153, -0.0856],
        [-0.0750, -0.1634, -0.2668,  0.0215, -0.3052,  0.0190],
        [-0.2076, -0.0629,  0.2162,  0.0605, -0.3636,  0.2171],
        [-0.0497, -0.4910, -0.0349,  0.1652, -0.0955, -0.0744],
        [-0.1695, -0.1348,  0.1497,  0.1119,  0.0312,  0.0198],
        [-0.1884, -0.1595,  0.1806,  0.1151,  0.0188, -0.0158],
        [ 0.0340,  0.0160,  0.0969, -0.0504, -0.1640,  0.0797],
        [ 0.0914, -0.3413,  0.0774,  0.0137, -0.2760,  0.0809],
        [-0.1778, -0.4239, -0.0323, -0.1510, -0.0646,  0.0191],
        [-0.0864, -0.0850,  0.0674, -0.0449,  0.0023,  0.1211],
        [-0.0667, -0.3690, -0.0694, -0.2

# Train the network

In [53]:
num_epoch = 1
iter_by_epoch = len(training_generator.dataset.list_IDs)/training_generator.batch_size 
eval_every = int(iter_by_epoch) #eval one time by epoch 
train_loss, train_accs = [], []
valid_loss, valid_accs = [], []
valid_iter = []
train_iter = []

def get_input_and_label(batch):
    inputs, labels = batch[0], batch[1] 
    # Change type
#     inputs = inputs.type(torch.float)
#     labels = labels.type(torch.long)
    # Transfer to GPU
    inputs, labels = inputs.to(device), labels.to(device)
    return(inputs, labels)

iter_by_epoch = len(training_generator.dataset.list_IDs)/training_generator.batch_size


for epoch in range(num_epoch):
    print("["+str(epoch)+"]"+"="*epoch)
    running_loss = 0.0
    net.train()
    # Training
    for i, data in enumerate(training_generator,0):
#         if i % eval_every == 0:
#             valid_iter.append(i + (1+epoch)*iter_by_epoch)
#             net.eval()
#             val_losses, val_accs, val_lengths = 0, 0, 0
            
#             for data in validation_generator:
#                 num = len(data[1]) #number of sequences in the batch
#                 inputs, labels = get_input_and_label(data)
#                 output = net(inputs)
#                 val_losses += criterion(output, labels)*num
#                 val_accs += accuracy(output, labels) *num
#                 val_lengths += num

#             # divide by the total accumulated batch sizes
#             val_losses /= val_lengths
#             val_accs /= val_lengths
#             valid_loss.append(get_numpy(val_losses))
#             valid_accs.append(get_numpy(val_accs))
#             net.train()
        inputs, labels = get_input_and_label(data) 

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]

        train_loss.append(get_numpy(loss))
        train_accs.append(get_numpy(accuracy(outputs, labels)))
        train_iter.append(i + (epoch+1)*iter_by_epoch)


fig = plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(train_iter, train_loss, label='train_loss')
plt.plot(valid_iter, valid_loss, label='valid_loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_iter, train_accs, label='train_accs')
plt.plot(valid_iter, valid_accs, label='valid_accs')
plt.legend()
plt.show()

print('Finished Training')

[0]
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])


/home/lea/Documents/DeepLearning/VenvDL/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100, 24984])
torch.Size([100, 6])
torch.Size([1, 100, 6246])
torch.Size([1, 100, 6246, 4])
torch.Size([100,

KeyboardInterrupt: 